<b>PLEASE NOTE THAT THIS NOTEBOOK WAS RUN IN GOOGLE COLAB AS SUCH CERTAIN IMPORT STATEMENTS MAY DIFFER</b>

In [0]:
!pip install catboost

In [0]:
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import math
from sklearn import metrics

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
train_path=os.path.abspath(os.path.join(os.path.dirname("__file__"), '..', '/content/drive/My Drive/datasets/train.csv'))
training_df = pd.read_csv(train_path, index_col="tripid")

In [0]:
training_df = training_df.replace({'label': {'incorrect': 0, 'correct' : 1}})

In [0]:
training_columns = ['duration','meter_waiting','meter_waiting_fare','fare','additional_fare']

In [0]:
target_column = ['label']

In [0]:
x = training_df[training_columns].values
y = training_df[target_column].values

In [0]:
x, x_test, y, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)

In [0]:
from catboost import CatBoostClassifier


model = CatBoostClassifier(verbose=True,
                           loss_function='CrossEntropy',
                           iterations=10000, 
                           eval_metric='F1',
                           devices='0:1')


In [0]:
model.fit(x,y)

In [0]:
predicted_y = model.predict(x_test)

In [0]:
print(); print(metrics.classification_report(y_test, predicted_y))
print(); print(metrics.confusion_matrix(y_test, predicted_y))


              precision    recall  f1-score   support

           0       0.80      0.62      0.70       504
           1       0.96      0.98      0.97      4649

    accuracy                           0.95      5153
   macro avg       0.88      0.80      0.84      5153
weighted avg       0.94      0.95      0.94      5153


[[ 313  191]
 [  78 4571]]


In [0]:

test_path =os.path.abspath(os.path.join(os.path.dirname("__file__"), '..', '/content/drive/My Drive/datasets/test.csv'))
test_set = pd.read_csv(test_path, index_col="tripid")



In [0]:
test_features = test_set[training_columns]

In [0]:
predicted_labels = model.predict(test_features)

In [0]:
sub_path =os.path.abspath(os.path.join(os.path.dirname("__file__"), '..', '/content/drive/My Drive/datasets/sample_submission.csv'))
submission_set = pd.read_csv(sub_path, index_col="tripid")

In [0]:
submission_set['prediction']= predicted_labels

In [0]:
import glob

filename = '/content/drive/My Drive/datasets/cat/cat_{%i}.csv'
dirname = '/content/drive/My Drive/datasets/cat/'
fileversion = 1

if not os.path.exists(dirname):
    os.makedirs(dirname)
while glob.glob(filename.replace('{%i}',str(fileversion))) :
    fileversion+=1
submission_set.to_csv(filename.replace('{%i}',str(fileversion)), index=True)
print("Completed!")



Completed!


In [0]:
submission_set['prediction'].idxmin()

213299545

In [0]:
submission_set['prediction'].value_counts()

1    8101
0     475
Name: prediction, dtype: int64